# Arbres de décisions classifieur from scratch

Dans ce notebook, vous allez coder votre propre algorithme de machine learning pour entraîner des arbres de décision afin de résoudre des problèmes de classification. 

Le code que vous trouverez dans ce notebook est fonctionnelle pour créer un algorithme d'apprentissage automatique. Attention, ce code n'est pas optimis, il est uniquement à but pédagogique pour que vous compreniez bien ce qui se passe lorsque vous entraînez un algorithme de machine learning d'arbres de décision. Je vais vous accompagner pas à pas dans l'implémentation de cet algorithme.

Si vous voulez entraîner un modèle à but professionnel, je vous recommande d'utiliser le package [Sklearn](https://scikit-learn.org/stable/index.html) qui contient toutes les librairies essentiel pour vous aider. 

Merci de soutenir ce travail de création de contenus en vous abonnant à la chaîne youtube AIForYou et en mettant une étoile au répertoire github. 



## Importation des packages

Comme cet algorithme est codé from scratch nous n'aurons pas besoin de beaucoup de librairies. Nous aurons seulement besoin de [Numpy](https://numpy.org/) pour gérer les calculs matriciels, [Pandas](https://pandas.pydata.org/) pour gérer les dataframe, et certains opérateurs de calcul. 

In [0]:
import numpy as np 
import pandas as pd
from functools import reduce
import operator

## Importation des données

Dans ce notebook, nous allons utiliser un dataframe homemade que j'ai créé pour vous. Le but sera de trouver la bonne classification des animaux en fonction de leurs caractéristiques. Nous allons utiliser 4 variables : si l'animal possède un gésier, le nombre de membres de l'animal, s'il l'animal possède des poils et si l'animal possède une carapace dorsal et ventral. 

Je me suis aidé de ce [site](http://soutien67.free.fr/svt/animaux/classification/classification01.htm) pour créer ce dataset.

In [0]:
dataframe = pd.DataFrame([['oiseau', 'coq', 1, 4, 0, 0],
                          ['oiseau', 'autruche', 1, 4, 0, 0],
                          ['oiseau', 'pigeon', 1, 4, 0, 0],
                          ['crocodilien', 'crocodile', 1, 4, 0, 0],
                          ['crocodilien', 'aligator', 1, 4, 0, 0],
                          ['Chélonien','tortue', 0, 4, 0, 1],
                          ['actynoptérygien','thon', 0, 0, 0, 0],
                          ['actynoptérygien','brochet', 0, 0, 0, 0],
                          ['mammifère','chien', 0, 4, 1, 0],
                          ['mammifère','chat', 0, 4, 1, 0],
                          ['mammifère','éléphant', 0, 4, 1, 0],
                          ['mammifère','sourie', 0, 4, 1, 0]],
                          columns=['classification', 'espèce', 'gésier', 'nb membres', 'poil', 'carapace dorsal et ventral'])

In [0]:
dataframe

,classification,espèce,gésier,nb membres,poil,carapace dorsal et ventral
0,oiseau,coq,1,4,0,0
1,oiseau,autruche,1,4,0,0
2,oiseau,pigeon,1,4,0,0
3,crocodilien,crocodile,1,4,0,0
4,crocodilien,aligator,1,4,0,0
5,Chélonien,tortue,0,4,0,1
6,actynoptérygien,thon,0,0,0,0
7,actynoptérygien,brochet,0,0,0,0
8,mammifère,chien,0,4,1,0
9,mammifère,chat,0,4,1,0


Nous n'aurons pas besoin de la variable 'espèce' qui est un identifiant dans ce dataframe.

In [0]:
dataframe_classif = dataframe.loc[:, ('classification', 'gésier', 'nb membres', 'poil', 'carapace dorsal et ventral')]
dataframe_classif

,classification,gésier,nb membres,poil,carapace dorsal et ventral
0,oiseau,1,4,0,0
1,oiseau,1,4,0,0
2,oiseau,1,4,0,0
3,crocodilien,1,4,0,0
4,crocodilien,1,4,0,0
5,Chélonien,0,4,0,1
6,actynoptérygien,0,0,0,0
7,actynoptérygien,0,0,0,0
8,mammifère,0,4,1,0
9,mammifère,0,4,1,0


## Création de la classe

La première étape de notre algorithme est la création de la classe qui va contenir toutes les fonctions dont notre algorithme à besoin pour fonctionner. On va commencer par la fonction de d'initialisation.

In [0]:
class decision_tree_classifier :
  """ Cette classe a pour but de créer un algorithme d'apprentissage automatique
  d'arbres de décision classifieur"""

  def __init__(self, target, dataframe, max_depth):
    """Cette fonction à pour but d'initialiser les variables essentiel à la 
    construction de notre algorithme.
   INPUT 
    - target : la variable cible qu'il faudra classifier
    - dataframe : les données d'apprentissage
    - max_deapth : la profondeur maximal de l'arbre à entraîner
   """
    self.max_depth = max_depth
    self.target = target
    self.dataframe = dataframe

## Séparation

Maintenant que nous avons notre classe, il va falloir implémenter toutes les fonctions nécessaires à l'entraînement d'un arbre de décision. On va commencer par la séparation d'une branche en deux sous-branches. 

On peut avoir à faire à deux types de variables : 
- Les variables quantitatives qui sont des variables réelles ;
- Les variables quantitatives qui sont des classes. 

In [0]:
class decision_tree_classifier :
  """ Cette classe a pour but de créer un algorithme d'apprentissage automatique
  d'arbres de décision classifieur"""

  def __init__(self, target, dataframe, max_depth):
    """Cette fonction a pour but d'initialiser les variables essentiel à la 
    construction de notre algorithme.
   INPUT 
    - target : la variable cible qu'il faudra classifier
    - dataframe : les données d'apprentissage
    - max_deapth : la profondeur maximal de l'arbre à entraîner
   """
    self.max_depth = max_depth
    self.target = target
    self.dataframe = dataframe

  def __quanti_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable quantitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer nore jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est plus petit ou égale à 'value'
    - right : dataframe avec les données où 'feature' est plus grande que 'value' 
   """
   
   left = dataset[dataset.loc[:,feature]<= value]
   right = dataset[dataset.loc[:,feature]> value]

   return left, right

  def __quali_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable qualitative'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer nore jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est égale 'value'
    - right : dataframe avec les données où 'feature' est différent 'value'
   """
   
   left = dataset[dataset.loc[:,feature]== value]
   right = dataset[dataset.loc[:,feature]!= value]

   return left, right

Je vous propose un exemple afin de bien comprendre comment la fonction fonctionne. Le but étant de séparer le dataframe initiale en deux dataframes distinct suivant si la variable 'poil' est égale à 0 ou non.

In [0]:
tree_classif = decision_tree_classifier('classification', dataframe_classif, 4)
left_classif, right_classif = tree_classif.__quali_split__('poil', 1, dataframe_classif)
print(left_classif)
print(right_classif)

   classification  gésier  nb membres  poil  carapace dorsal et ventral
8       mammifère       0           4     1                           0
9       mammifère       0           4     1                           0
10      mammifère       0           4     1                           0
11      mammifère       0           4     1                           0
    classification  gésier  nb membres  poil  carapace dorsal et ventral
0           oiseau       1           4     0                           0
1           oiseau       1           4     0                           0
2           oiseau       1           4     0                           0
3      crocodilien       1           4     0                           0
4      crocodilien       1           4     0                           0
5        Chélonien       0           4     0                           1
6  actynoptérygien       0           0     0                           0
7  actynoptérygien       0           0     0            

## Fonction de coût

Maintenant que l'on sait comment séparer notre jeu de données, nous allons trouver un moyen de calculer le coût de cette séparation. Le but étant de trouver la séparation correspondant au coût le plus faible. 

Pour calculer à quel point notre séparation est optimale nous allons utiliser l'indice Gini. Pour ceux qui ne connaissent pas cette métrique, je vous recommande de regarder ma [vidéo sur la théorie des arbres de décision](https://www.youtube.com/watch?v=gxS_R1Ph9ak).

Le but est de créer des sous-groupes les plus purs possible.

In [0]:
# Nos targets values de départ
row = dataframe_classif['classification']
print(row)

0              oiseau
1              oiseau
2              oiseau
3         crocodilien
4         crocodilien
5           Chélonien
6     actynoptérygien
7     actynoptérygien
8           mammifère
9           mammifère
10          mammifère
11          mammifère
Name: classification, dtype: object


On utilise les informations calculées ci-dessus afin de calculer l'indice Gini.

$$ I_G(f) = 1 - \sum^m_{i-1}f_i^2$$

où $f_i$ est la probabilité d'appartenance à la classe $i$.

On va commencer par créer un dictionnaire avec chaque classe et son itération (ici 1 pour chaque classe).

In [0]:
class_dict = list(map(lambda x: {x: 1}, row))
print(class_dict)

[{'oiseau': 1}, {'oiseau': 1}, {'oiseau': 1}, {'crocodilien': 1}, {'crocodilien': 1}, {'Chélonien': 1}, {'actynoptérygien': 1}, {'actynoptérygien': 1}, {'mammifère': 1}, {'mammifère': 1}, {'mammifère': 1}, {'mammifère': 1}]


On va regrouper ces itérations afin d'obtenir un dictionnaire avec en clef chaque classe et en valeur le nombre d'itérations de cette classe dans le groupe.

In [0]:
def add_dict(prec_dict, new_dict) :
  """ Cette fonction fusionne des dictionnaires 
  en sommant les valeurs des clefs similaires
  INPUT 
     - prec_dict : dictionnaire de fusion
     - new_dict : dictionnaire à fusionner """
  if list(new_dict.keys())[0] in prec_dict:
    prec_dict[list(new_dict.keys())[0]] += 1
  else : 
    prec_dict[list(new_dict.keys())[0]] = 1
  return prec_dict

In [0]:
class_dict_sum = reduce(add_dict, class_dict)
print(class_dict_sum)

{'oiseau': 3, 'crocodilien': 2, 'Chélonien': 1, 'actynoptérygien': 2, 'mammifère': 4}


On extrait le vecteur des occurrences du dictionnaire.

In [0]:
occu_class = np.fromiter(reduce(add_dict, list(map(lambda x: {x: 1}, dataframe_classif['classification']))).values(), dtype=float)
print(occu_class)

[3. 2. 1. 2. 4.]


On calcul le nombre total de populations dans notre classe.

In [0]:
pop = np.sum(occu_class)
print(pop)

12.0


On calcule la probabilité d'appartenance à chaque classe.

In [0]:
print(occu_class/pop)

[0.25       0.16666667 0.08333333 0.16666667 0.33333333]


On utilise les informations calculées ci-dessus afin de calculer l'indice Gini.

$$ I_G(f) = 1 - \sum^m_{i-1}f_i^2$$

où $f_i$ est la probabilité d'appartenance à la classe $i$.

In [0]:
impurity = 1 - np.sum((occu_class/pop)**2)
print(impurity)

0.7638888888888888


In [0]:
class decision_tree :
  """ Cette classe a pour but de créer un algorithme d'apprentissage automatique
  d'arbres de décision classifieur"""

  def __init__(self, target, dataframe, max_depth):
    """Cette fonction a pour but d'initialiser les variables essentiel à la 
    construction de notre algorithme.
   INPUT 
    - target : la variable cible qu'il faudra classifier
    - dataframe : les données d'apprentissage
    - max_deapth : la profondeur maximal de l'arbre à entraîner
   """
    self.max_depth = max_depth
    self.target = target
    self.dataframe = dataframe

  def __quanti_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable quantitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer notre jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est plus petit ou égale à 'value'
    - right : dataframe avec les données où 'feature' est plus grande que 'value' 
   """
   
   left = dataset[dataset.loc[:,feature]<= value]
   right = dataset[dataset.loc[:,feature]> value]

   return left, right

  def __quali_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable qualitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer notre jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est égale 'value'
    - right : dataframe avec les données où 'feature' est différent 'value'
   """
   
   left = dataset[dataset.loc[:,feature]== value]
   right = dataset[dataset.loc[:,feature]!= value]

   return left, right


  def __add_dict__(self, prec_dict, new_dict) :
    """ Cette fonction fusionne des dictionnaires 
    en sommant les valeurs des clefs similaires
    INPUT 
       - prec_dict : dictionnaire de fusion
       - new_dict : dictionnaire à fusionner """
    if list(new_dict.keys())[0] in prec_dict:
      prec_dict[list(new_dict.keys())[0]] += 1
    else : 
      prec_dict[list(new_dict.keys())[0]] = 1
    return prec_dict

  def __gini__(self, dataset):
    """Calculer les indices Gini du dataset passé en paramètre
    INPUT 
       - dataset : dataframe contenant la variable 'self.target'
    OUTPUT
       - impurity : l'impureté calculée à partir du dataset
    """
    rows = dataset[self.target]
    class_dict = list(map(lambda x: {x: 1}, rows))
    class_dict_sum = reduce(self.__add_dict__, class_dict)
    occu_class = np.fromiter(reduce(self.__add_dict__, 
                                    list(map(lambda x: {x: 1}, rows))).values(), 
                             dtype=float)
    pop = np.sum(occu_class)
    impurity = 1 - np.sum((occu_class/pop)**2)
    return impurity   

Voici un exemple pour vérifier que la fonction '\_\_gini\_\_' fonctionne correctement.

In [0]:
tree_classif = decision_tree('classification', dataframe_classif, 4)
tree_classif.__gini__(dataframe_classif)

0.7638888888888888

## Évaluer une séparation

Félicitations, vous avez maintenant votre fonction '\_\_gini\_\_' pour calculer les performances d'un nœud . Maintenant, vous allez pouvoir créer une fonction capable de déterminer le coût d'une séparation d'un nœud  en deux branches.

In [0]:
class decision_tree :
  """ Cette classe a pour but de créer un algorithme d'apprentissage automatique
  d'arbres de décision classifieur"""
  
  def __init__(self, target, dataframe, max_depth):
    """Cette fonction a pour but d'initialiser les variables essentiel à la 
    construction de notre algorithme.
   INPUT 
    - target : la variable cible qu'il faudra classifier
    - dataframe : les données d'apprentissage
    - max_deapth : la profondeur maximal de l'arbre à entraîner
   """
    self.max_depth = max_depth
    self.target = target
    self.dataframe = dataframe

  def __quanti_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable quantitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer notre jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est plus petit ou égale à 'value'
    - right : dataframe avec les données où 'feature' est plus grande que 'value' 
   """
   
   left = dataset[dataset.loc[:,feature]<= value]
   right = dataset[dataset.loc[:,feature]> value]

   return left, right

  def __quali_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable qualitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer notre jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est égale 'value'
    - right : dataframe avec les données où 'feature' est différent 'value'
   """
   
   left = dataset[dataset.loc[:,feature]== value]
   right = dataset[dataset.loc[:,feature]!= value]

   return left, right


  def __add_dict__(self, prec_dict, new_dict) :
    """ Cette fonction fusionne des dictionnaires 
    en sommant les valeurs des clefs similaires
    INPUT 
       - prec_dict : dictionnaire de fusion
       - new_dict : dictionnaire à fusionner 
    OUTPUT 
       - prec_dict : dictionnaire avec en clef la 'classe'
       et en 'valeur' son occurence dans le dataset"""
    if list(new_dict.keys())[0] in prec_dict:
      prec_dict[list(new_dict.keys())[0]] += 1
    else : 
      prec_dict[list(new_dict.keys())[0]] = 1
    return prec_dict

  def __gini__(self, dataset):
    """Calculer les indices Gini du dataset passé en paramètre
    INPUT 
       - dataset : dataframe contenant la variable 'self.target'
    OUTPUT
       - impurity : l'impureté calculée à partir du dataset
    """
    rows = dataset[self.target]
    class_dict = list(map(lambda x: {x: 1}, rows))
    class_dict_sum = reduce(self.__add_dict__, class_dict)
    occu_class = np.fromiter(reduce(self.__add_dict__, 
                                    list(map(lambda x: {x: 1}, rows))).values(), 
                             dtype=float)
    pop = np.sum(occu_class)
    impurity = 1 - np.sum((occu_class/pop)**2)
    return impurity   

  def __split_evaluator__(self, left_dataset, right_dataset):
    """ Calculer le coût d'une séparation d'un noeud en deux branches
    INPUT 
       - left_dataset : dataset de la branche de gauche
       - right_dataset : dataset de la branche de droite
    OUTPUT 
       - cost : coût de la séparation"""
    left_eval = self.__gini__(left_dataset)
    nb_left = left_dataset.shape[0]
    right_eval = self.__gini__(right_dataset)
    nb_right = right_dataset.shape[0]
    nb_tot = nb_left + nb_right
    cost = nb_left/nb_tot * left_eval + nb_right/nb_tot * right_eval
    return cost 

Un exemple pour vérifier que la fonction d'évaluation de la séparation fonctionne bien.

In [0]:
tree_classif = decision_tree('classification', dataframe_classif, 4)
left_classif, right_classif = tree_classif.__quali_split__('poil', 1, dataframe_classif)
tree_classif.__split_evaluator__(left_classif, right_classif)

0.47916666666666663

## Évaluer les séparations possibles

Maintenant que l'on sait évaluer la performance des séparations, il faut créer une fonction qui va tester toutes les séparations possibles afin de les évaluer et trouver la meilleure séparation. 

In [0]:
class decision_tree :
  """ Cette classe a pour but de créer un algorithme d'apprentissage automatique
  d'arbres de décision classifieur"""

  def __init__(self, target, dataframe, max_depth):
    """Cette fonction a pour but d'initialiser les variables essentiel à la 
    construction de notre algorithme.
   INPUT 
    - target : la variable cible qu'il faudra classifier
    - dataframe : les données d'apprentissage
    - max_deapth : la profondeur maximal de l'arbre à entraîner
   """
    self.max_depth = max_depth
    self.target = target
    self.dataframe = dataframe

  def __quanti_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable quantitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer notre jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est plus petit ou égale à 'value'
    - right : dataframe avec les données où 'feature' est plus grande que 'value' 
   """
   
   left = dataset[dataset.loc[:,feature]<= value]
   right = dataset[dataset.loc[:,feature]> value]

   return left, right

  def __quali_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable qualitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer notre jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est égale 'value'
    - right : dataframe avec les données où 'feature' est différent 'value'
   """
   
   left = dataset[dataset.loc[:,feature]== value]
   right = dataset[dataset.loc[:,feature]!= value]

   return left, right


  def __add_dict__(self, prec_dict, new_dict) :
    """ Cette fonction fusionne des dictionnaires 
    en sommant les valeurs des clefs similaires
    INPUT 
       - prec_dict : dictionnaire de fusion
       - new_dict : dictionnaire à fusionner 
    OUTPUT 
       - prec_dict : dictionnaire avec en clef la 'classe'
       et en 'valeur' son occurence dans le dataset"""
    if list(new_dict.keys())[0] in prec_dict:
      prec_dict[list(new_dict.keys())[0]] += 1
    else : 
      prec_dict[list(new_dict.keys())[0]] = 1
    return prec_dict

  def __gini__(self, dataset):
    """Calculer les indices Gini du dataset passé en paramètre
    INPUT 
       - dataset : dataframe contenant la variable 'self.target'
    OUTPUT
       - impurity : l'impureté calculée à partir du dataset
    """
    rows = dataset[self.target]
    class_dict = list(map(lambda x: {x: 1}, rows))
    class_dict_sum = reduce(self.__add_dict__, class_dict)
    occu_class = np.fromiter(reduce(self.__add_dict__, 
                                    list(map(lambda x: {x: 1}, rows))).values(), 
                             dtype=float)
    pop = np.sum(occu_class)
    impurity = 1 - np.sum((occu_class/pop)**2)
    return impurity   

  def __split_evaluator__(self, left_dataset, right_dataset):
    """ Calculer le coût d'une séparation d'un noeud en deux branches
    INPUT 
       - left_dataset : dataset de la branche de gauche
       - right_dataset : dataset de la branche de droite
    OUTPUT 
       - cost : coût de la séparation"""
    left_eval = self.__gini__(left_dataset)
    nb_left = left_dataset.shape[0]
    right_eval = self.__gini__(right_dataset)
    nb_right = right_dataset.shape[0]
    nb_tot = nb_left + nb_right
    cost = nb_left/nb_tot * left_eval + nb_right/nb_tot * right_eval
    return cost 

  def __test_quali__(self, dataset, feature):
    """ Tester toutes les séparations possibles d'une variable qualitative
    INPUT 
       - dataset : dataset à évaluer
       - feature : variable du dataset à évaluer
    OUTPUT 
       - df_eval : dataframe contenant le coût de chaque séparation"""   

    df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
    for value in dataset.loc[:, feature].unique() :
      left, right = self.__quali_split__(feature, value, dataset)
      cost_result = self.__split_evaluator__(left, right)
      df_eval = df_eval.append(pd.DataFrame([[feature, 
                                              value,
                                              'quali',
                                              cost_result]],
                                            columns=('feature', 'value', 'nature', 'cost')))
    return df_eval


  def __test_quanti__(self, dataset, feature):
    """ Tester toutes les séparations possibles d'une variable quantitative
    INPUT 
       - dataset : dataset à évaluer
       - feature : variable du dataset à évaluer
    OUTPUT 
       - df_eval : dataframe contenant le coût de chaque séparation""" 

    df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
    value_to_test = (dataset.loc[:, feature].sort_values()[1:].values + dataset.loc[:, feature].sort_values()[:-1].values)/2
    for value in value_to_test :
      left, right = self.__quanti_split__(feature, value, dataset)
      cost_result = self.__split_evaluator__(left, right)
      df_eval = df_eval.append(pd.DataFrame([[feature, 
                                              value,
                                              'quanti',
                                              cost_result]],
                                            columns=('feature', 'value', 'nature', 'cost')))
    return df_eval

Un exemple pour vérifier notre nouvelle fonction.

In [0]:
tree_classif = decision_tree('classification', dataframe_classif, 4)
tree_classif.__test_quali__(dataframe_classif, 'gésier')

,feature,value,nature,cost
0,gésier,1,quali,0.533333
0,gésier,0,quali,0.533333


Utilisons nos nouvelles fonctions afin de déterminer la meilleure séparation de notre dataset.

In [0]:
class decision_tree :
  """ Cette classe a pour but de créer un algorithme d'apprentissage automatique
  d'arbres de décision classifieur"""

  def __init__(self, target, dataframe, max_depth):
    """Cette fonction a pour but d'initialiser les variables essentiel à la 
    construction de notre algorithme.
   INPUT 
    - target : la variable cible qu'il faudra classifier
    - dataframe : les données d'apprentissage
    - max_deapth : la profondeur maximal de l'arbre à entraîner
   """
    self.max_depth = max_depth
    self.target = target
    self.dataframe = dataframe

  def __quanti_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable quantitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer notre jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est plus petit ou égale à 'value'
    - right : dataframe avec les données où 'feature' est plus grande que 'value' 
   """
   
   left = dataset[dataset.loc[:,feature]<= value]
   right = dataset[dataset.loc[:,feature]> value]

   return left, right

  def __quali_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable qualitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer notre jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est égale 'value'
    - right : dataframe avec les données où 'feature' est différent 'value'
   """
   
   left = dataset[dataset.loc[:,feature]== value]
   right = dataset[dataset.loc[:,feature]!= value]

   return left, right


  def __add_dict__(self, prec_dict, new_dict) :
    """ Cette fonction fusionne des dictionnaires 
    en sommant les valeurs des clefs similaires
    INPUT 
       - prec_dict : dictionnaire de fusion
       - new_dict : dictionnaire à fusionner 
    OUTPUT 
       - prec_dict : dictionnaire avec en clef la 'classe'
       et en 'valeur' son occurence dans le dataset"""
    if list(new_dict.keys())[0] in prec_dict:
      prec_dict[list(new_dict.keys())[0]] += 1
    else : 
      prec_dict[list(new_dict.keys())[0]] = 1
    return prec_dict

  def __gini__(self, dataset):
    """Calculer les indices Gini du dataset passé en paramètre
    INPUT 
       - dataset : dataframe contenant la variable 'self.target'
    OUTPUT
       - impurity : l'impureté calculée à partir du dataset
    """
    rows = dataset[self.target]
    class_dict = list(map(lambda x: {x: 1}, rows))
    class_dict_sum = reduce(self.__add_dict__, class_dict)
    occu_class = np.fromiter(reduce(self.__add_dict__, 
                                    list(map(lambda x: {x: 1}, rows))).values(), 
                             dtype=float)
    pop = np.sum(occu_class)
    impurity = 1 - np.sum((occu_class/pop)**2)
    return impurity   

  def __split_evaluator__(self, left_dataset, right_dataset):
    """ Calculer le coût d'une séparation d'un noeud en deux branches
    INPUT 
       - left_dataset : dataset de la branche de gauche
       - right_dataset : dataset de la branche de droite
    OUTPUT 
       - cost : coût de la séparation"""
    left_eval = self.__gini__(left_dataset)
    nb_left = left_dataset.shape[0]
    right_eval = self.__gini__(right_dataset)
    nb_right = right_dataset.shape[0]
    nb_tot = nb_left + nb_right
    cost = nb_left/nb_tot * left_eval + nb_right/nb_tot * right_eval
    return cost 

  def __test_quali__(self, dataset, feature):
    """ Tester toutes les séparations possibles d'une variable qualitative
    INPUT 
       - dataset : dataset à évaluer
       - feature : variable du dataset à évaluer
    OUTPUT 
       - df_eval : dataframe contenant le coût de chaque séparation"""   

    df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
    for value in dataset.loc[:, feature].unique() :
      left, right = self.__quali_split__(feature, value, dataset)
      cost_result = self.__split_evaluator__(left, right)
      df_eval = df_eval.append(pd.DataFrame([[feature, 
                                              value,
                                              'quali',
                                              cost_result]],
                                            columns=('feature', 'value', 'nature', 'cost')))
    return df_eval


  def __test_quanti__(self, dataset, feature):
    """ Tester toutes les séparations possibles d'une variable quantitative
    INPUT 
       - dataset : dataset à évaluer
       - feature : variable du dataset à évaluer
    OUTPUT 
       - df_eval : dataframe contenant le coût de chaque séparation""" 

    df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
    value_to_test = (dataset.loc[:, feature].sort_values()[1:].values + dataset.loc[:, feature].sort_values()[:-1].values)/2
    for value in value_to_test :
      left, right = self.__quanti_split__(feature, value, dataset)
      cost_result = self.__split_evaluator__(left, right)
      df_eval = df_eval.append(pd.DataFrame([[feature, 
                                              value,
                                              'quanti',
                                              cost_result]],
                                            columns=('feature', 'value', 'nature', 'cost')))
    return df_eval


  def __find_best_split__(self, dataset):
    """ Trouver la meilleure séparation de notre jeu de données
    INPUT 
       - dataset : jeu de données à séparer
    OUTPUT 
       - def_eval : dataset contenant 'feature' variable à séparer, 'value' 
       la valeur à laquelle séparer la variable, 'nature' la nature de la 
       variable et 'cost' le coût de cette séparation"""
    df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
    columns = dataset.columns[np.logical_not(dataset.columns == self.target)]
    for column in columns : 
      if len(dataset[column].unique()) >= 10 :
        df_eval = df_eval.append(self.__test_quanti__(dataset, column))
      elif len(dataset[column].unique()) > 1 :
        df_eval = df_eval.append(self.__test_quali__(dataset, column))

    df_eval = df_eval.reset_index(drop=True)

    idx_cost_min = df_eval['cost'].idxmin(axis=0, skipna=True)

    return df_eval.iloc[idx_cost_min, :]

Un exemple, pour vérifier si notre fonction est opérationnelle. 

In [0]:
tree_classif = decision_tree('classification', dataframe_classif, 4)
tree_classif.__find_best_split__(dataframe_classif)

feature        poil
value             0
nature        quali
cost       0.479167
Name: 4, dtype: object

## Les branches

Nous avons rassemblé quasiment toutes les fonctions nécessaires pour créer notre algorithme d'arbre de décision. Il nous manque cependant la création de branches qui constitue le corps de notre arbre. Nous allons représenter nos branches sous forme de classe. 

In [0]:
class node:
   """Cette classe a pour but de représenter les branches de notre arbre de
   classification.
   """   
   def __init__(self, feature, value, cost, nature, left_branch, right_branch, depth, pop):
      self.feature = feature
      self.value = value
      self.nature = nature
      self.left_branch = left_branch
      self.right_branch = right_branch
      self.depth = depth
      self.pop = pop  

   def __split__(self):
      if self.nature == 'quanti' :
          return self.feature + ' <= ' + str(self.value)
      elif self.nature == 'quali' :
          return self.feature + ' == ' + str(self.value)

Un exemple pour vérifier que notre classe conserve les informations dont nous avons besoin.

In [0]:
tree_classif = decision_tree('classification', dataframe_classif, 4)
feature, value, nature, cost = tree_classif.__find_best_split__(dataframe_classif)
left_branch, right_branch = tree_classif.__quali_split__(feature, value, dataframe_classif)
branch = node(feature, value, cost, nature, left_branch, right_branch, 0, 100)

print(branch.__split__())

poil == 0


## Les feuilles 

Nous avons rassemblé quasiment toutes les fonctions nécessaires pour créer notre algorithme d'arbre de décision. Il nous manque cependant la création de feuille. Nous allons représenter nos feuilles sous forme de classe. 

In [0]:
class decision_tree :
  """ Cette classe a pour but de créer un algorithme d'apprentissage automatique
  d'arbres de décision classifieur"""

  def __init__(self, target, dataframe, max_depth):
    """Cette fonction a pour but d'initialiser les variables essentiel à la 
    construction de notre algorithme.
   INPUT 
    - target : la variable cible qu'il faudra classifier
    - dataframe : les données d'apprentissage
    - max_deapth : la profondeur maximal de l'arbre à entraîner
   """
    self.max_depth = max_depth
    self.target = target
    self.dataframe = dataframe

  def __quanti_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable quantitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer notre jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est plus petit ou égale à 'value'
    - right : dataframe avec les données où 'feature' est plus grande que 'value' 
   """
   
   left = dataset[dataset.loc[:,feature]<= value]
   right = dataset[dataset.loc[:,feature]> value]

   return left, right

  def __quali_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable qualitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer notre jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est égale 'value'
    - right : dataframe avec les données où 'feature' est différent 'value'
   """
   
   left = dataset[dataset.loc[:,feature]== value]
   right = dataset[dataset.loc[:,feature]!= value]

   return left, right


  def __add_dict__(self, prec_dict, new_dict) :
    """ Cette fonction fusionne des dictionnaires 
    en sommant les valeurs des clefs similaires
    INPUT 
       - prec_dict : dictionnaire de fusion
       - new_dict : dictionnaire à fusionner 
    OUTPUT 
       - prec_dict : dictionnaire avec en clef la 'classe'
       et en 'valeur' son occurence dans le dataset"""
    if list(new_dict.keys())[0] in prec_dict:
      prec_dict[list(new_dict.keys())[0]] += 1
    else : 
      prec_dict[list(new_dict.keys())[0]] = 1
    return prec_dict

  def __gini__(self, dataset):
    """Calculer les indices Gini du dataset passé en paramètre
    INPUT 
       - dataset : dataframe contenant la variable 'self.target'
    OUTPUT
       - impurity : l'impureté calculée à partir du dataset
    """
    rows = dataset[self.target]
    class_dict = list(map(lambda x: {x: 1}, rows))
    class_dict_sum = reduce(self.__add_dict__, class_dict)
    occu_class = np.fromiter(reduce(self.__add_dict__, 
                                    list(map(lambda x: {x: 1}, rows))).values(), 
                             dtype=float)
    pop = np.sum(occu_class)
    impurity = 1 - np.sum((occu_class/pop)**2)
    return impurity   

  def __split_evaluator__(self, left_dataset, right_dataset):
    """ Calculer le coût d'une séparation d'un noeud en deux branches
    INPUT 
       - left_dataset : dataset de la branche de gauche
       - right_dataset : dataset de la branche de droite
    OUTPUT 
       - cost : coût de la séparation"""
    left_eval = self.__gini__(left_dataset)
    nb_left = left_dataset.shape[0]
    right_eval = self.__gini__(right_dataset)
    nb_right = right_dataset.shape[0]
    nb_tot = nb_left + nb_right
    cost = nb_left/nb_tot * left_eval + nb_right/nb_tot * right_eval
    return cost 

  def __test_quali__(self, dataset, feature):
    """ Tester toutes les séparations possibles d'une variable qualitative
    INPUT 
       - dataset : dataset à évaluer
       - feature : variable du dataset à évaluer
    OUTPUT 
       - df_eval : dataframe contenant le coût de chaque séparation"""   

    df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
    for value in dataset.loc[:, feature].unique() :
      left, right = self.__quali_split__(feature, value, dataset)
      cost_result = self.__split_evaluator__(left, right)
      df_eval = df_eval.append(pd.DataFrame([[feature, 
                                              value,
                                              'quali',
                                              cost_result]],
                                            columns=('feature', 'value', 'nature', 'cost')))
    return df_eval


  def __test_quanti__(self, dataset, feature):
    """ Tester toutes les séparations possibles d'une variable quantitative
    INPUT 
       - dataset : dataset à évaluer
       - feature : variable du dataset à évaluer
    OUTPUT 
       - df_eval : dataframe contenant le coût de chaque séparation""" 

    df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
    value_to_test = (dataset.loc[:, feature].sort_values()[1:].values + dataset.loc[:, feature].sort_values()[:-1].values)/2
    for value in value_to_test :
      left, right = self.__quanti_split__(feature, value, dataset)
      cost_result = self.__split_evaluator__(left, right)
      df_eval = df_eval.append(pd.DataFrame([[feature, 
                                              value,
                                              'quanti',
                                              cost_result]],
                                            columns=('feature', 'value', 'nature', 'cost')))
    return df_eval


  def __find_best_split__(self, dataset):
    """ Trouver la meilleure séparation de notre jeu de données
    INPUT 
       - dataset : jeu de données à séparer
    OUTPUT 
       - def_eval : dataset contenant 'feature' variable à séparer, 'value' 
       la valeur à laquelle séparer la variable, 'nature' la nature de la 
       variable et 'cost' le coût de cette séparation"""
    df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
    columns = dataset.columns[np.logical_not(dataset.columns == self.target)]
    for column in columns : 
      if len(dataset[column].unique()) >= 10 :
        df_eval = df_eval.append(self.__test_quanti__(dataset, column))
      elif len(dataset[column].unique()) > 1 :
        df_eval = df_eval.append(self.__test_quali__(dataset, column))

    df_eval = df_eval.reset_index(drop=True)

    idx_cost_min = df_eval['cost'].idxmin(axis=0, skipna=True)

    return df_eval.iloc[idx_cost_min, :]

  def create_leaf(self, dataset):
    """ Création d'une feuille 
    INPUT 
       - dataset : dataset de la feuille à construire
    OUTPUT 
       - leaf : la classe feuille créée avec les informations de notre dataset"""

    labels = dataset[self.target]
    pop = labels.shape[0]
    class_dict = list(map(lambda x: {x: 1}, labels))
    class_dict_sum = reduce(self.__add_dict__, class_dict)
    prediction = max(class_dict_sum.items(), key=operator.itemgetter(1))[0]
    proba = {k: v/pop for k, v in class_dict_sum.items()}
  
    return leaf(dataset, pop, class_dict_sum, prediction, proba)

class leaf:
  """Cette classe a pour but de représenter les feuilles de notre arbre de
  classification.
  """
  def __init__(self, dataset, pop, class_dict_sum, prediction, proba):
    self.dataset = dataset
    self.pop = pop
    self.class_dict_sum = class_dict_sum
    self.prediction = prediction
    self.proba = proba

Un exemple pour vérifier que notre classe conserve les informations dont nous avons besoin.

In [0]:
tree_classif = decision_tree('classification', dataframe_classif, 4)
leaf_classif = tree_classif.create_leaf(dataframe_classif)
print(leaf_classif.prediction)
print(leaf_classif.pop)
print(leaf_classif.proba)

mammifère
12
{'oiseau': 0.25, 'crocodilien': 0.16666666666666666, 'Chélonien': 0.08333333333333333, 'actynoptérygien': 0.16666666666666666, 'mammifère': 0.3333333333333333}


## Entraînement

Maintenant que nous avons créé toutes nos fonctions et nos classes, on peut passer à l'implémentation de notre fonction d'entraînement.

In [0]:
class decision_tree :
  """ Cette classe a pour but de créer un algorithme d'apprentissage automatique
  d'arbres de décision classifieur"""

  def __init__(self, target, dataframe, max_depth):
    """Cette fonction a pour but d'initialiser les variables essentiel à la 
    construction de notre algorithme.
   INPUT 
    - target : la variable cible qu'il faudra classifier
    - dataframe : les données d'apprentissage
    - max_deapth : la profondeur maximal de l'arbre à entraîner
   """
    self.max_depth = max_depth
    self.target = target
    self.dataframe = dataframe

  def __quanti_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable quantitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer notre jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est plus petit ou égale à 'value'
    - right : dataframe avec les données où 'feature' est plus grande que 'value' 
   """
   
   left = dataset[dataset.loc[:,feature]<= value]
   right = dataset[dataset.loc[:,feature]> value]

   return left, right

  def __quali_split__(self, feature, value, dataset):
   """Cette fonction split un jeu de données en fonction
   de la valeur 'value' de la variable qualitative 'feature' passé en paramètre
   INPUT 
    - feature : integer correspondant à la variable à séparer
    - value : integer correspond à la valeur à laquelle séparer notre jeu de données
    - dataset : pandas dataframe à séparer
   OUTPUT 
    - left : dataframe avec les données où 'feature' est égale 'value'
    - right : dataframe avec les données où 'feature' est différent 'value'
   """
   
   left = dataset[dataset.loc[:,feature]== value]
   right = dataset[dataset.loc[:,feature]!= value]

   return left, right


  def __add_dict__(self, prec_dict, new_dict) :
    """ Cette fonction fusionne des dictionnaires 
    en sommant les valeurs des clefs similaires
    INPUT 
       - prec_dict : dictionnaire de fusion
       - new_dict : dictionnaire à fusionner 
    OUTPUT 
       - prec_dict : dictionnaire avec en clef la 'classe'
       et en 'valeur' son occurence dans le dataset"""
    if list(new_dict.keys())[0] in prec_dict:
      prec_dict[list(new_dict.keys())[0]] += 1
    else : 
      prec_dict[list(new_dict.keys())[0]] = 1
    return prec_dict

  def __gini__(self, dataset):
    """Calculer les indices Gini du dataset passé en paramètre
    INPUT 
       - dataset : dataframe contenant la variable 'self.target'
    OUTPUT
       - impurity : l'impureté calculée à partir du dataset
    """
    rows = dataset[self.target]
    class_dict = list(map(lambda x: {x: 1}, rows))
    class_dict_sum = reduce(self.__add_dict__, class_dict)
    occu_class = np.fromiter(reduce(self.__add_dict__, 
                                    list(map(lambda x: {x: 1}, rows))).values(), 
                             dtype=float)
    pop = np.sum(occu_class)
    impurity = 1 - np.sum((occu_class/pop)**2)
    return impurity   

  def __split_evaluator__(self, left_dataset, right_dataset):
    """ Calculer le coût d'une séparation d'un noeud en deux branches
    INPUT 
       - left_dataset : dataset de la branche de gauche
       - right_dataset : dataset de la branche de droite
    OUTPUT 
       - cost : coût de la séparation"""
    left_eval = self.__gini__(left_dataset)
    nb_left = left_dataset.shape[0]
    right_eval = self.__gini__(right_dataset)
    nb_right = right_dataset.shape[0]
    nb_tot = nb_left + nb_right
    cost = nb_left/nb_tot * left_eval + nb_right/nb_tot * right_eval
    return cost 

  def __test_quali__(self, dataset, feature):
    """ Tester toutes les séparations possibles d'une variable qualitative
    INPUT 
       - dataset : dataset à évaluer
       - feature : variable du dataset à évaluer
    OUTPUT 
       - df_eval : dataframe contenant le coût de chaque séparation"""   

    df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
    for value in dataset.loc[:, feature].unique() :
      left, right = self.__quali_split__(feature, value, dataset)
      cost_result = self.__split_evaluator__(left, right)
      df_eval = df_eval.append(pd.DataFrame([[feature, 
                                              value,
                                              'quali',
                                              cost_result]],
                                            columns=('feature', 'value', 'nature', 'cost')))
    return df_eval


  def __test_quanti__(self, dataset, feature):
    """ Tester toutes les séparations possibles d'une variable quantitative
    INPUT 
       - dataset : dataset à évaluer
       - feature : variable du dataset à évaluer
    OUTPUT 
       - df_eval : dataframe contenant le coût de chaque séparation""" 

    df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
    value_to_test = (dataset.loc[:, feature].sort_values()[1:].values + dataset.loc[:, feature].sort_values()[:-1].values)/2
    for value in value_to_test :
      left, right = self.__quanti_split__(feature, value, dataset)
      cost_result = self.__split_evaluator__(left, right)
      df_eval = df_eval.append(pd.DataFrame([[feature, 
                                              value,
                                              'quanti',
                                              cost_result]],
                                            columns=('feature', 'value', 'nature', 'cost')))
    return df_eval


  def __find_best_split__(self, dataset):
    """ Trouver la meilleure séparation de notre jeu de données
    INPUT 
       - dataset : jeu de données à séparer
    OUTPUT 
       - def_eval : dataset contenant 'feature' variable à séparer, 'value' 
       la valeur à laquelle séparer la variable, 'nature' la nature de la 
       variable et 'cost' le coût de cette séparation"""
    df_eval = pd.DataFrame([], columns=('feature', 'value', 'nature', 'cost'))
    columns = dataset.columns[np.logical_not(dataset.columns == self.target)]
    for column in columns : 
      if len(dataset[column].unique()) >= 10 :
        df_eval = df_eval.append(self.__test_quanti__(dataset, column))
      elif len(dataset[column].unique()) > 1 :
        df_eval = df_eval.append(self.__test_quali__(dataset, column))

    df_eval = df_eval.reset_index(drop=True)

    idx_cost_min = df_eval['cost'].idxmin(axis=0, skipna=True)

    return df_eval.iloc[idx_cost_min, :]

  def create_leaf(self, dataset):
    """ Création d'une feuille 
    INPUT 
       - dataset : dataset de la feuille à construire
    OUTPUT 
       - leaf : la classe feuille créé avec les informations de notre dataset"""

    labels = dataset[self.target]
    pop = labels.shape[0]
    class_dict = list(map(lambda x: {x: 1}, labels))
    class_dict_sum = reduce(self.__add_dict__, class_dict)
    prediction = max(class_dict_sum.items(), key=operator.itemgetter(1))[0]
    proba = {k: v/pop for k, v in class_dict_sum.items()}
  
    return leaf(dataset, pop, class_dict_sum, prediction, proba)

  def training(self, dataset, depth=0):
      """Cette fonction va construire l'arbre de décision en fonction des 
      paramètres fournir à l'initialisation de cette classe.
      INPUT 
         - depth : profondeur actuel de l'arbre
      OUTPUT 
         - node : racine de l'arbre
      """

      # Cette partie de code vérifie que le dataset peut encore être séparé
      no_more_split = True
      columns = dataset.columns[np.logical_not(dataset.columns == self.target)]
      for column in columns : 
        if len(dataset[column].unique()) > 1 :
          no_more_split = False

      # Si le dataset est pure, ou que la profondeur maximum est atteinte ou
      # que le dataset ne peut plus être séparé nous créons une feuille
      if len(dataset[self.target].unique())==1 or depth==self.max_depth or no_more_split:
        return self.create_leaf(dataset)

      # Recherche de la meilleur séparation
      split_eval = self.__find_best_split__(dataset)

      # Si le coût obtenu après séparation est moins bon le coût actuel, 
      # création d'une feuille avec le dataset actuel
      if split_eval['cost'] >= self.__gini__(dataset) :
          return self.create_leaf(dataset)

      # Séparation du dataset selon la nature de la variable choisie
      if split_eval['nature'] == 'quali' :
         left_branch, right_branch = self.__quali_split__(split_eval['feature'], split_eval['value'], dataset)
      elif split_eval['nature'] == 'quanti' :
         left_branch, right_branch = self.__quanti_split__(split_eval['feature'], split_eval['value'], dataset)

      # Entraînement récursif de la branche de gauche
      left_node = self.training(left_branch, depth+1)

      # Entraînement récursif de la branche de droite
      right_node = self.training(right_branch, depth+1)

      # On retourne la racine de l'arbre
      return node(split_eval['feature'], 
                  split_eval['value'], 
                  split_eval['cost'], 
                  split_eval['nature'],
                  left_node,
                  right_node,
                  depth,
                  dataset.shape[0])
      
  def fit(self):
    """ Entraînement du modèle
    OUTPUT 
       - node : racine de l'arbre"""
    return self.training(self.dataframe)

Vérification pour voir si la variable fonctionne correctement.

In [0]:
tree_classif = decision_tree('classification', dataframe_classif, 4)
tree_trained = tree_classif.fit()

## Visualisation de l'arbre

Notre arbre est construit, mais il est compliqué de le comprendre, nous allons donc construire une fonction qui va nous permettre de visualiser l'arbre.

In [0]:
def print_tree(node, spacing=""):
  """Affichage de l'arbre de décision
  INPUT 
     - node : branche à afficher
     - spacings : espace à afficher en fonction de la profondeur de la branche"""

  # Différents affichages si c'est une feuille 
  if isinstance(node, leaf):
      print (spacing + "Predict", node.prediction)
      print (spacing + "Predict", node.proba)
      return

  # Affichage de la condition de la séparation
  print (spacing + node.__split__())

  # Dans le cas où la condition est vérifiée
  print (spacing + '--> True:')
  print_tree(node.left_branch, spacing + "  ")

  # Dans le cas où la condition n'est pas vérifiée
  print (spacing + '--> False:')
  print_tree(node.right_branch, spacing + "  ")

In [0]:
dataframe_classif

,classification,gésier,nb membres,poil,carapace dorsal et ventral
0,oiseau,1,4,0,0
1,oiseau,1,4,0,0
2,oiseau,1,4,0,0
3,crocodilien,1,4,0,0
4,crocodilien,1,4,0,0
5,Chélonien,0,4,0,1
6,actynoptérygien,0,0,0,0
7,actynoptérygien,0,0,0,0
8,mammifère,0,4,1,0
9,mammifère,0,4,1,0


In [0]:
tree_classif = decision_tree('classification', dataframe_classif, 4)
tree_trained = tree_classif.fit()
print_tree(tree_trained)

poil == 0
--> True:
  nb membres == 4
  --> True:
    gésier == 1
    --> True:
      Predict oiseau
      Predict {'oiseau': 0.6, 'crocodilien': 0.4}
    --> False:
      Predict Chélonien
      Predict {'Chélonien': 1.0}
  --> False:
    Predict actynoptérygien
    Predict {'actynoptérygien': 1.0}
--> False:
  Predict mammifère
  Predict {'mammifère': 1.0}


J'espère que ce notebook vous a plu, abonnez-vous à la chaîne YouTube et mettez une étoile sur le github de ce répertoire.

## Sources : 
- https://www.youtube.com/watch?v=gxS_R1Ph9ak&t=548s
- http://soutien67.free.fr/svt/animaux/classification/classification01.htm
- https://www.youtube.com/watch?v=LDRbO9a6XPU
- https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/
- https://www.youtube.com/watch?v=g9c66TUylZ4
- https://www.youtube.com/watch?v=7VeUPuFGJHk&t=599s
